In [1]:
# import library 
import sktime 
from sktime.utils.load_data import load_from_tsfile_to_dataframe

In [2]:
from sktime.transformers.compose import ColumnConcatenator
from sktime.classifiers.compose import TimeSeriesForestClassifier
from sktime.classifiers.dictionary_based.boss import BOSSEnsemble
from sktime.classifiers.compose import ColumnEnsembleClassifier
from sktime.classifiers.shapelet_based import ShapeletTransformClassifier
from sktime.datasets import load_basic_motions
from sktime.pipeline import Pipeline

import numpy as np
import pandas as pd

In [3]:
from sktime.highlevel.tasks import TSCTask
from sktime.highlevel.strategies import TSCStrategy

from sktime.transformers.compose import RowwiseTransformer
from sktime.transformers.compose import ColumnTransformer
from sktime.transformers.compose import Tabulariser
from sktime.transformers.segment import RandomIntervalSegmenter

from sktime.pipeline import Pipeline
from sktime.pipeline import FeatureUnion
from sktime.classifiers.distance_based import ProximityForest

from sktime.classifiers.compose import TimeSeriesForestClassifier
from sktime.classifiers.distance_based import KNeighborsTimeSeriesClassifier

from sktime.datasets import load_gunpoint
from sktime.utils.time_series import time_series_slope

from statsmodels.tsa.stattools import acf
from statsmodels.tsa.ar_model import AR

from sklearn.preprocessing import FunctionTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


In [4]:
from sktime.utils.load_data import load_from_arff_to_dataframe

#split dataset 
Xtrain1, ytrain1 = load_from_arff_to_dataframe("../sktime-notebooks/data/Multivariate2018_arff/Multivariate_arff/AtrialFibrillation/AtrialFibrillationDimension1_TRAIN.arff")
Xtrain2, ytrain2 = load_from_arff_to_dataframe("../sktime-notebooks/data/Multivariate2018_arff/Multivariate_arff/AtrialFibrillation/AtrialFibrillationDimension2_TRAIN.arff")
Xtest1, ytest1 = load_from_arff_to_dataframe("../sktime-notebooks/data/Multivariate2018_arff/Multivariate_arff/AtrialFibrillation/AtrialFibrillationDimension1_TEST.arff")
Xtest2, ytest2 = load_from_arff_to_dataframe("../sktime-notebooks/data/Multivariate2018_arff/Multivariate_arff/AtrialFibrillation/AtrialFibrillationDimension2_TEST.arff")

#combined dataset 
Xtrain, ytrain = load_from_arff_to_dataframe("../sktime-notebooks/data/Multivariate2018_arff/Multivariate_arff/AtrialFibrillation/AtrialFibrillation_TRAIN.arff")
Xtest, ytest = load_from_arff_to_dataframe("../sktime-notebooks/data/Multivariate2018_arff/Multivariate_arff/AtrialFibrillation/AtrialFibrillation_TEST.arff")



In [5]:
Xtrain1.head()

,dim_0
0,0 -0.34086 1 -0.38038 2 -0.34580 3...
1,0 -0.11362 1 -0.07410 2 -0.05928 3...
2,0 -0.2079 1 -0.1683 2 -0.1980 3 ...
3,0 -0.11805 1 -0.08657 2 -0.09444 3...
4,0 -0.11362 1 -0.06422 2 -0.05928 3...


In [6]:
Xtrain2.head()

,dim_0
0,0 0.14820 1 0.13338 2 0.10868 3...
1,0 -0.00988 1 -0.02470 2 -0.00494 3...
2,0 -0.02632 1 -0.04606 2 -0.08554 3...
3,0 0.03510 1 0.04680 2 0.06435 3...
4,0 -0.04940 1 0.01482 2 0.03952 3...


In [7]:
Xtrain.head()

,dim_0,dim_1
0,0 -0.34086 1 -0.38038 2 -0.34580 3...,0 0.14820 1 0.13338 2 0.10868 3...
1,0 -0.11362 1 -0.07410 2 -0.05928 3...,0 -0.00988 1 -0.02470 2 -0.00494 3...
2,0 -0.2079 1 -0.1683 2 -0.1980 3 ...,0 -0.02632 1 -0.04606 2 -0.08554 3...
3,0 -0.11805 1 -0.08657 2 -0.09444 3...,0 0.03510 1 0.04680 2 0.06435 3...
4,0 -0.11362 1 -0.06422 2 -0.05928 3...,0 -0.04940 1 0.01482 2 0.03952 3...


## How to merge multiiple .arff data

In [8]:
Xtrain2_renamed=Xtrain2.rename(columns={'dim_0':'dim_1'})

In [9]:
myconcat=[Xtrain1,Xtrain2_renamed]
true_X=pd.concat(myconcat, axis=1)

In [10]:
true_X.head()

,dim_0,dim_1
0,0 -0.34086 1 -0.38038 2 -0.34580 3...,0 0.14820 1 0.13338 2 0.10868 3...
1,0 -0.11362 1 -0.07410 2 -0.05928 3...,0 -0.00988 1 -0.02470 2 -0.00494 3...
2,0 -0.2079 1 -0.1683 2 -0.1980 3 ...,0 -0.02632 1 -0.04606 2 -0.08554 3...
3,0 -0.11805 1 -0.08657 2 -0.09444 3...,0 0.03510 1 0.04680 2 0.06435 3...
4,0 -0.11362 1 -0.06422 2 -0.05928 3...,0 -0.04940 1 0.01482 2 0.03952 3...


In [11]:
np.unique(ytrain)

array(['n', 's', 't'], dtype='<U1')

In [12]:
Xtrain

,dim_0,dim_1
0,0 -0.34086 1 -0.38038 2 -0.34580 3...,0 0.14820 1 0.13338 2 0.10868 3...
1,0 -0.11362 1 -0.07410 2 -0.05928 3...,0 -0.00988 1 -0.02470 2 -0.00494 3...
2,0 -0.2079 1 -0.1683 2 -0.1980 3 ...,0 -0.02632 1 -0.04606 2 -0.08554 3...
3,0 -0.11805 1 -0.08657 2 -0.09444 3...,0 0.03510 1 0.04680 2 0.06435 3...
4,0 -0.11362 1 -0.06422 2 -0.05928 3...,0 -0.04940 1 0.01482 2 0.03952 3...
5,0 -0.02964 1 0.00988 2 0.00988 3...,0 0.00000 1 -0.00988 2 -0.02964 3...
6,0 0.66445 1 1.16450 2 1.00010 3...,0 0.02684 1 0.08723 2 -0.16775 3...
7,0 0.07686 1 0.06588 2 0.02196 3...,0 0.13090 1 0.13090 2 0.09520 3...
8,0 -0.2091 1 -0.2091 2 -0.2040 3 ...,0 0.01104 1 -0.00552 2 0.00000 3...
9,0 -0.01482 1 -0.03458 2 -0.03458 3...,0 0.01482 1 0.01482 2 0.00494 3...


In [13]:
ytrain

array(['n', 'n', 'n', 'n', 'n', 's', 's', 's', 's', 's', 't', 't', 't',
       't', 't'], dtype='<U1')

In [35]:
fullDatasetLabels=np.concatenate((ytest,ytrain), axis =0)
shuffled=np.random.permutation(fullDatasetLabels.shape[0])
fullDatasetLabelsShuffled=fullDatasetLabels[shuffled]
shuffled

array([23,  9, 15, 11, 21, 18,  2, 29, 19, 24,  0, 14,  4, 17,  1, 10, 28,
        5, 16,  3, 25, 22,  7, 27,  6, 26,  8, 12, 20, 13])

In [16]:
fullDatasetLabelsShuffled

array(['t', 't', 't', 't', 's', 'n', 's', 'n', 'n', 'n', 't', 't', 's',
       'n', 's', 't', 'n', 'n', 's', 't', 'n', 'n', 'n', 's', 't', 's',
       's', 't', 's', 's'], dtype='<U1')

In [36]:
fullDataset=pd.concat([Xtrain,Xtest])
fullDataset=fullDataset.reset_index()
fullDataset=fullDataset.reindex(shuffled)
fullDataset=fullDataset.drop(columns=['index'])
fullDataset

,dim_0,dim_1
23,0 -0.08398 1 -0.08398 2 -0.06916 3...,0 -0.01482 1 0.00000 2 -0.02964 3...
9,0 -0.01482 1 -0.03458 2 -0.03458 3...,0 0.01482 1 0.01482 2 0.00494 3...
15,0 0.41990 1 0.35568 2 0.25688 3...,0 -0.05928 1 -0.03952 2 -0.01976 3...
11,0 0.05434 1 0.00988 2 0.03952 3...,0 0.00988 1 0.01976 2 0.00988 3...
21,0 -0.05391 1 -0.06589 2 -0.08386 3...,0 0.27166 1 0.30056 2 0.31790 3...
18,0 -0.06422 1 -0.04446 2 -0.06422 3...,0 0.00988 1 0.01976 2 0.02964 3...
2,0 -0.2079 1 -0.1683 2 -0.1980 3 ...,0 -0.02632 1 -0.04606 2 -0.08554 3...
29,0 -0.07137 1 -0.01647 2 -0.01098 3...,0 0.10115 1 0.08925 2 0.08925 3...
19,0 -0.11264 1 -0.09152 2 -0.07744 3...,0 -0.00699 1 -0.00699 2 -0.00699 3...
24,0 -0.05928 1 -0.04446 2 -0.05434 3...,0 0.00988 1 0.00494 2 0.00988 3...


In [40]:
train_y_modified= fullDatasetLabelsShuffled[0:24]
test_y_modified= fullDatasetLabelsShuffled[25:30]

In [41]:
train_y_modified

array(['s', 's', 'n', 't', 's', 'n', 'n', 't', 'n', 's', 'n', 't', 'n',
       'n', 'n', 't', 't', 's', 'n', 'n', 't', 's', 's', 't'], dtype='<U1')

In [42]:
test_y_modified

array(['t', 's', 't', 's', 't'], dtype='<U1')

In [43]:
train_X_modified= fullDataset[0:24]
test_X_modifed= fullDataset[25:30]

In [44]:
test_X_modifed

,dim_0,dim_1
26,0 -0.1224 1 -0.1326 2 -0.1326 3 ...,0 -0.01104 1 -0.02760 2 0.01104 3...
8,0 -0.2091 1 -0.2091 2 -0.2040 3 ...,0 0.01104 1 -0.00552 2 0.00000 3...
12,0 0.02964 1 -0.06916 2 -0.10374 3...,0 0.18278 1 0.12844 2 0.10374 3...
20,0 -0.15314 1 -0.10374 2 -0.04940 3...,0 0.00988 1 0.01482 2 0.04446 3...
13,0 0.22762 1 0.29351 2 0.20366 3...,0 -0.93636 1 -0.43350 2 -0.13294 3...


In [49]:
steps = [
    ('concatenate', ColumnConcatenator()),
    ('classify', TimeSeriesForestClassifier(n_estimators=5))]
clf = Pipeline(steps)
clf.fit(train_X_modified, train_y_modified)
clf.score(test_X_modifed, test_y_modified)

0.0

In [56]:
steps = [
    ('concatenate', ColumnConcatenator()),
    ('classify', TimeSeriesForestClassifier(n_estimators=5))]
clf = Pipeline(steps)
clf.fit(Xtrain, ytrain)
clf.score(Xtest, ytest)

0.4

In [57]:
clf = ColumnEnsembleClassifier(estimators=[
    ("TSF0", TimeSeriesForestClassifier(n_estimators=5), [0]),
    ("BOSSEnsemble3", BOSSEnsemble(ensemble_size=5), [1]),
])
clf.fit(Xtrain, ytrain)
clf.score(Xtest, ytest)

0.2

In [59]:
clf = ShapeletTransformClassifier(time_contract_in_mins=0.5)
clf.fit(Xtrain, ytrain)
clf.score(Xtest, ytest)

0.4666666666666667